202001267 - Meet Bhatt <br>
202003039 - Mukund Ladani <br>
202003040 - Kashyap Halavadia

In [ ]:
pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install qtcodes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Introduction to repetition code

### The basics of error correction

In [ ]:
p1 = 0.01
p3 = 3 * p1**2 * (1-p1) + p1**3 # probability of 2 or 3 errors
print('Probability of a single reply being garbled: {}'.format(p1))
print('Probability of a the majority of three replies being garbled: {:.4f}'.format(p3))

Probability of a single reply being garbled: 0.01
Probability of a the majority of three replies being garbled: 0.0003


### Correcting errors in qubit

In [ ]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

def get_noise(p_meas,p_gate):

    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = depolarizing_error(p_gate, 1)
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["x"]) # single qubit gate error is applied to x gates
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
        
    return noise_model

In [ ]:
noise_model = get_noise(0.01,0.01)

In [ ]:
from qiskit import QuantumCircuit, execute, Aer

qc0 = QuantumCircuit(3,3,name='0') # initialize circuit with three qubits in the 0 state

qc0.measure(qc0.qregs[0],qc0.cregs[0]) # measure the qubits

# run the circuit with the noise model and extract the counts
counts = execute( qc0, Aer.get_backend('qasm_simulator'),noise_model=noise_model).result().get_counts()

print(counts)

{'100': 9, '001': 7, '010': 10, '000': 998}


In [ ]:
qc1 = QuantumCircuit(3, 3, name='0') # initialize circuit with three qubits in the 0 state
qc1.x(qc1.qregs[0]) # flip each 0 to 1

qc1.measure(qc1.qregs[0],qc1.cregs[0]) # measure the qubits

# run the circuit with th noise model and extract the counts
counts = execute(qc1, Aer.get_backend('qasm_simulator'),noise_model=noise_model).result().get_counts()

print(counts)

{'101': 13, '011': 19, '111': 978, '110': 14}


In [ ]:
noise_model = get_noise(0.5,0.0)
counts = execute(qc1, Aer.get_backend('qasm_simulator'),noise_model=noise_model).result().get_counts()
print(counts)

{'100': 119, '110': 135, '101': 123, '111': 113, '011': 137, '001': 144, '010': 129, '000': 124}


### Storing qubit

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister

cq = QuantumRegister(2, 'code_qubit')
lq = QuantumRegister(1, 'ancilla_qubit')
sb = ClassicalRegister(1, 'syndrome_bit')
qc = QuantumCircuit(cq, lq, sb)
qc.cx(cq[0], lq[0])
qc.cx(cq[1], lq[0])
qc.measure(lq, sb)
qc.draw()

code_qubit_0: ──■──────────
                  │          
  code_qubit_1: ──┼────■─────
                ┌─┴─┐┌─┴─┐┌─┐
 ancilla_qubit: ┤ X ├┤ X ├┤M├
                └───┘└───┘└╥┘
syndrome_bit: 1/═══════════╩═
                           0

In [ ]:
qc_init = QuantumCircuit(cq, lq, sb)

qc_init.compose(qc).draw()

code_qubit_0: ──■──────────
                  │          
  code_qubit_1: ──┼────■─────
                ┌─┴─┐┌─┴─┐┌─┐
 ancilla_qubit: ┤ X ├┤ X ├┤M├
                └───┘└───┘└╥┘
syndrome_bit: 1/═══════════╩═
                           0

In [ ]:
counts = execute(qc_init.compose(qc), Aer.get_backend('qasm_simulator')).result().get_counts()
print('Results:', counts)

Results: {'0': 1024}


In [ ]:
qc_init = QuantumCircuit(cq, lq, sb)
qc_init.x(cq)

qc_init.compose(qc).draw()

┌───┐             
  code_qubit_0: ┤ X ├──■──────────
                ├───┤  │          
  code_qubit_1: ┤ X ├──┼────■─────
                └───┘┌─┴─┐┌─┴─┐┌─┐
 ancilla_qubit: ─────┤ X ├┤ X ├┤M├
                     └───┘└───┘└╥┘
syndrome_bit: 1/════════════════╩═
                                0

In [ ]:
counts = execute(qc_init.compose(qc), Aer.get_backend('qasm_simulator')).result().get_counts()
print('Results:', counts)

Results: {'0': 1024}


In [ ]:
qc_init = QuantumCircuit(cq, lq, sb)
qc_init.h(cq[0])
qc_init.cx(cq[0], cq[1])

qc_init.compose(qc).draw()

┌───┐                  
  code_qubit_0: ┤ H ├──■────■──────────
                └───┘┌─┴─┐  │          
  code_qubit_1: ─────┤ X ├──┼────■─────
                     └───┘┌─┴─┐┌─┴─┐┌─┐
 ancilla_qubit: ──────────┤ X ├┤ X ├┤M├
                          └───┘└───┘└╥┘
syndrome_bit: 1/═════════════════════╩═
                                     0

In [ ]:
counts = execute(qc_init.compose(qc), Aer.get_backend('qasm_simulator')).result().get_counts()
print('Results:', counts)

Results: {'0': 1024}


In [ ]:
qc_init = QuantumCircuit(cq, lq, sb)
qc_init.h(cq[0])
qc_init.cx(cq[0],cq[1])
qc_init.x(cq[0])

qc_init.compose(qc).draw()

┌───┐     ┌───┐             
  code_qubit_0: ┤ H ├──■──┤ X ├──■──────────
                └───┘┌─┴─┐└───┘  │          
  code_qubit_1: ─────┤ X ├───────┼────■─────
                     └───┘     ┌─┴─┐┌─┴─┐┌─┐
 ancilla_qubit: ───────────────┤ X ├┤ X ├┤M├
                               └───┘└───┘└╥┘
syndrome_bit: 1/══════════════════════════╩═
                                          0

In [ ]:
counts = execute(qc_init.compose(qc), Aer.get_backend('qasm_simulator')).result().get_counts()
print('Results:',counts)

Results: {'1': 1024}


### Quantum repetition code

In [ ]:
from qiskit import *

In [ ]:
pip install qiskit-ignis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from qiskit.ignis.verification.topological_codes import RepetitionCode
from qiskit.ignis.verification.topological_codes import GraphDecoder
from qiskit.ignis.verification.topological_codes import lookuptable_decoding, postselection_decoding

<ipython-input-53-19f403370059>:1: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.verification.topological_codes import RepetitionCode


In [ ]:
n = 3
T = 1

code = RepetitionCode(n, T)

In [ ]:
code.circuit['0'].draw()

┌───┐     ┌───┐        ┌─┐         
      link_qubit_0: ┤ X ├─────┤ X ├────────┤M├─────────
                    └─┬─┘┌───┐└─┬─┘┌───┐   └╥┘┌─┐      
      link_qubit_1: ──┼──┤ X ├──┼──┤ X ├────╫─┤M├──────
                      │  └─┬─┘  │  └─┬─┘┌─┐ ║ └╥┘      
      code_qubit_0: ──■────┼────┼────┼──┤M├─╫──╫───────
                           │    │    │  └╥┘ ║  ║ ┌─┐   
      code_qubit_1: ───────■────■────┼───╫──╫──╫─┤M├───
                                     │   ║  ║  ║ └╥┘┌─┐
      code_qubit_2: ─────────────────■───╫──╫──╫──╫─┤M├
                                         ║  ║  ║  ║ └╥┘
round_0_link_bit: 2/═════════════════════╬══╩══╩══╬══╬═
                                         ║  0  1  ║  ║ 
        code_bit: 3/═════════════════════╩════════╩══╩═
                                         0        1  2

In [ ]:
code.circuit['1'].draw()

┌───┐     ┌───┐        ┌─┐         
      link_qubit_0: ─────┤ X ├─────┤ X ├────────┤M├─────────
                         └─┬─┘┌───┐└─┬─┘┌───┐   └╥┘┌─┐      
      link_qubit_1: ───────┼──┤ X ├──┼──┤ X ├────╫─┤M├──────
                    ┌───┐  │  └─┬─┘  │  └─┬─┘┌─┐ ║ └╥┘      
      code_qubit_0: ┤ X ├──■────┼────┼────┼──┤M├─╫──╫───────
                    ├───┤       │    │    │  └╥┘ ║  ║ ┌─┐   
      code_qubit_1: ┤ X ├───────■────■────┼───╫──╫──╫─┤M├───
                    ├───┤                 │   ║  ║  ║ └╥┘┌─┐
      code_qubit_2: ┤ X ├─────────────────■───╫──╫──╫──╫─┤M├
                    └───┘                     ║  ║  ║  ║ └╥┘
round_0_link_bit: 2/══════════════════════════╬══╩══╩══╬══╬═
                                              ║  0  1  ║  ║ 
        code_bit: 3/══════════════════════════╩════════╩══╩═
                                              0        1  2

In [ ]:
def get_raw_results(code,noise_model=None):
    circuits = code.get_circuit_list()
    raw_results = {}
    for log in range(2):
        job = execute( circuits[log], Aer.get_backend('qasm_simulator'), noise_model=noise_model)
        raw_results[str(log)] = job.result().get_counts(str(log))
    return raw_results

raw_results = get_raw_results(code)
for log in raw_results:
    print(f'Logical {log}: {raw_results[log]}')

Logical 0: {'000 00': 1024}
Logical 1: {'111 00': 1024}


In [ ]:
code = RepetitionCode(n, 4)

raw_results = get_raw_results(code)
for log in raw_results:
    print(f'Logical {log}: {raw_results[log]}')

Logical 0: {'000 00 00 00 00': 1024}
Logical 1: {'111 00 00 00 00': 1024}


In [ ]:
code = RepetitionCode(5, 4)

raw_results = get_raw_results(code)
for log in raw_results:
    print(f'Logical {log}: {raw_results[log]}')

Logical 0: {'00000 0000 0000 0000 0000': 1024}
Logical 1: {'11111 0000 0000 0000 0000': 1024}


### Lookup table decoding

In [ ]:
code = RepetitionCode(3, 1)

noise_model = get_noise(0.05, 0.05)

raw_results = get_raw_results(code,noise_model)
for log in raw_results:
    print(f'Logical {log}: {raw_results[log]}\n')

Logical 0: {'011 01': 1, '110 01': 1, '010 11': 1, '011 00': 5, '010 10': 6, '101 00': 2, '000 00': 634, '000 10': 86, '110 11': 1, '100 00': 47, '001 00': 46, '110 00': 3, '101 01': 1, '000 01': 81, '100 10': 5, '010 00': 57, '001 10': 4, '110 10': 1, '010 01': 19, '001 01': 6, '100 01': 10, '000 11': 7}

Logical 1: {'000 10': 1, '010 10': 1, '110 11': 1, '100 00': 6, '011 11': 2, '010 01': 2, '001 01': 1, '100 01': 6, '110 10': 3, '011 01': 14, '101 10': 8, '110 01': 21, '111 10': 57, '011 10': 21, '011 00': 52, '101 00': 53, '111 01': 72, '001 11': 2, '111 11': 4, '110 00': 52, '101 01': 17, '100 10': 2, '111 00': 599, '101 11': 17, '010 00': 3, '001 10': 3, '001 00': 4}



In [ ]:
circuits = code.get_circuit_list()
table_results = {}
for log in range(2):
    job = execute( circuits[log], Aer.get_backend('qasm_simulator'), noise_model=noise_model, shots=10000 )
    table_results[str(log)] = job.result().get_counts(str(log))

In [ ]:
P = lookuptable_decoding(raw_results, table_results)
print('P =', P)

P = {'0': 0.0, '1': 0.0}


### Graph theoretic decoding

In [ ]:
code = RepetitionCode(3, 2)

raw_results = get_raw_results(code, noise_model)

results = code.process_results(raw_results)

for log in ['0', '1']:
    print(f'\nLogical {log}:')
    print('raw results       ', {string:raw_results[log][string] for string in raw_results[log] if raw_results[log][string]>=50 })
    print('processed results ', {string:results[log][string] for string in results[log] if results[log][string]>=50 })


Logical 0:
raw results        {'000 01 00': 53, '000 10 10': 54, '000 00 00': 498}
processed results  {'0 0  00 01 01': 53, '0 0  10 10 00': 54, '0 0  00 00 00': 498}

Logical 1:
raw results        {'111 00 00': 468, '111 01 01': 50}
processed results  {'1 1  00 00 00': 468, '1 1  01 01 00': 50}
